# Diffusion 1

## Inference 3: Concepts (Textual Inversion & Dreambooth)

## Workflow

#### Drive

If you need to load/save to your drive:

```python
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive/')

import os
os.chdir('drive/My Drive/IS53055B-DMLCP/DMLCP/python') # to change to another directory
```

#### Huggingface login

This time, we will definitely log into a Huggingface account. If you haven't, create an account [here](https://huggingface.co/) and then to ['/settings/tokens'](https://huggingface.co/settings/tokens) to create an access token.


#### Install


1. On Colab, just use `pip` to install Huggingface libraries (see below).

2. Locally, the install is the same as the one used for Language models, see [`setup.md`](https://github.com/jchwenger/DMLCP/blob/main/setup.md#pytorch--huggingfacegradio).

In [ ]:
import sys

if "google.colab" in sys.modules:
    !pip install -Uq transformers diffusers accelerate

In [ ]:
from pathlib import Path
from huggingface_hub import notebook_login
if not (Path.home()/'.huggingface'/'token').exists():
    notebook_login()

## 1. Textual Inversion

Adapted from the [Huggingface page](https://huggingface.co/docs/diffusers/training/text_inversion) \(see also the the [fast.ai repo](https://github.com/fastai/diffusion-nbs)\).

In [ ]:
#@title Install and import requirements

from PIL import Image
import matplotlib.pyplot as plt

import torch

# Get cpu, gpu or mps device for training.
# See: https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html#creating-models
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

from diffusers import StableDiffusionPipeline

The convenience function to display a batch of images.

In [ ]:
# https://matplotlib.org/stable/gallery/axes_grid1/simple_axesgrid.html
from mpl_toolkits.axes_grid1 import ImageGrid

def plot_images(imgs, rows=1, cols=None, figsize=(12,8), title=None):
    fig = plt.figure(figsize=figsize)           # control figure size
    grid = ImageGrid(
        fig, 111,                                                     # similar to subplot(111) | see: https://stackoverflow.com/a/11404223
        nrows_ncols=(rows, cols if cols is not None else len(imgs)),  # control rows/cols
        axes_pad=0.1,                                                 # pad between axes in inch
    )
    if title is not None:               # https://matplotlib.org/3.2.1/gallery/subplots_axes_and_figures/figure_title.html
        fig.suptitle(title, x=0, y=0.5)

    for ax, im in zip(grid, imgs):      # Iterating over the grid returns the Axes.
        ax.set_xticks([])               # no x/y ticks: https://stackoverflow.com/a/45149018
        ax.set_yticks([])               #               https://stackoverflow.com/a/58535290
        ax.imshow(im)

### Concept library [here](https://huggingface.co/sd-concepts-library).

You can find the name of the concept in the file `token_identifier.txt`.

Additional example found [here](https://huggingface.co/sd-concepts-library/collage-style3).

In [ ]:
repo_id_embeds = "sd-concepts-library/collage-style3" # original example: "sd-concepts-library/cat-toy"

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    use_safetensors=True
).to(device)

pipe.load_textual_inversion(repo_id_embeds)

In [ ]:
# prompt = "a grafitti in a NYC wall with a <cat-toy> on it"
prompt = "<collage_style> with buildings, flowers, and books"

num_samples = 2
num_rows = 2

all_images = []
for _ in range(num_rows):
    images = pipe(
        prompt,
        num_images_per_prompt=num_samples,
        num_inference_steps=50,
        guidance_scale=7.5
    ).images
    all_images.extend(images)

plot_images(all_images)

---

## 2. Dreambooth

Adapted from [this notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/sd_dreambooth_inference.ipynb#scrollTo=DXBVVJjl_lwt). See also [the documentation](https://huggingface.co/docs/diffusers/training/dreambooth).

I recommend restarting your runtime if you want to test this part!

In [ ]:
#@title Install and import requirements

import sys

if "google.colab" in sys.modules:
    !pip install -qqq diffusers==0.11.1 transformers gradio ftfy accelerate

import gradio as gr

from PIL import Image

import torch
from torch import autocast

# Get cpu, gpu or mps device for training.
# See: https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html#creating-models
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

from diffusers import StableDiffusionPipeline

The convenience function to display a batch of images.

In [ ]:
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
#@title Login to the Hugging Face Hub
#@markdown Optional step, do it if you want to run private concepts
from huggingface_hub import notebook_login
!git config --global credential.helper store

notebook_login()

In [ ]:
#@title Load the model from the [Concepts Library](https://huggingface.co/sd-dreambooth-library). If you are new to Stable Diffusion, make sure you [read the LICENSE](https://github.com/CompVis/stable-diffusion/blob/main/LICENSE)!
#@markdown  You may also use a locally trained model by replacing the `MODEL_ID` to a path with the model locally or on a mounted Google Drive

#@markdown  Additional example found [here](https://huggingface.co/sd-dreambooth-library/noggles-glasses-1200)

# MODEL_ID = "sd-dreambooth-library/cat-toy" # original
MODEL_ID = "sd-dreambooth-library/noggles-glasses-1200" #@param {type:"string"}

pipe = StableDiffusionPipeline.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16
).to(device)

In [ ]:
#@title Run the Stable Diffusion pipeline with interactive UI Demo on Gradio
#@markdown Run this cell to get a Gradio UI like this to run your models

#@markdown ![](https://i.imgur.com/bxHfawQ.png)

def inference(prompt, num_samples):
    all_images = []
    images = pipe(
        prompt,
        num_images_per_prompt=num_samples,
        num_inference_steps=50,
        guidance_scale=7.5
    ).images
    all_images.extend(images)
    return all_images

with gr.Blocks() as demo:
    gr.HTML("<h2 style=\"font-size: 2em; font-weight: bold\" align=\"center\">Stable Diffusion Dreambooth - Run Concept</h2>")
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="prompt")
            samples = gr.Slider(label="Samples",value=1)
            run = gr.Button(value="Run")
        with gr.Column():
            gallery = gr.Gallery(show_label=False)

    run.click(inference, inputs=[prompt,samples], outputs=gallery)
    gr.Examples(
        # [["a photo of sks toy riding a bicycle", 1,1]], # original
        [["a cat wearing sks glasses", 1]],
        [prompt,samples], gallery, inference, cache_examples=False)

demo.launch(debug=True)

In [ ]:
#@title Run the Stable Diffusion pipeline on Colab
#@markdown Don't forget to use the `sks` token in your prompt

# prompt = "a photo of sks toy floating on a ramen bowl" #@param {type:"string"}
prompt = "a cat wearing sks glasses" #@param {type:"string"}

num_samples = 2 #@param {type:"number"}
num_rows = 2 #@param {type:"number"}

all_images = []
for _ in range(num_rows):
    images = pipe(
        prompt,
        num_images_per_prompt=num_samples,
        num_inference_steps=50,
        guidance_scale=7.5
    ).images
    all_images.extend(images)

grid = image_grid(all_images, num_samples, num_rows)
grid